In [1]:
!pip install vidgear
!pip install  opencv-python ultralytics openfilter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:



import torch
import numpy as np
import cv2
from ultralytics import YOLO # Import the YOLO class for YOLOv8

from openfilter.filter_runtime import Frame, Filter
from openfilter.filter_runtime.filters.video_in import VideoIn
from openfilter.filter_runtime.filters.video_out import VideoOutConfig, VideoOut

class YOLOv8PersonCounterFilter(Filter):
    def setup(self, config):
        """
        Initializes the YOLOv8 model and sets up class IDs for detection.
        This method is called once when the filter pipeline starts.
        """
        print(f'YOLOv8PersonCounterFilter setup: {config.my_option=}')

        # Load a pre-trained YOLOv8 model.
        # 'yolov8n.pt' is the nano version, good for real-time and edge devices.
        self.model = YOLO('yolov8n.pt')
        self.model.fuse() # Fuse model for faster inference


        self.class_names_dict = self.model.names
        print("YOLOv8 Class Names:", self.class_names_dict)

        # Find the class ID for 'person' by iterating through the dictionary's items.
        # This is the CORRECT way to get the key (ID) from a value ('person') in a dictionary.
        for class_id, class_name in self.class_names_dict.items():
            if class_name == 'person':
                self.person_class_id = class_id
            elif class_name == 'car':
                self.car_class_id = class_id
                break



        # Set a confidence threshold for detections.
        self.confidence_threshold = 0.5

    def process(self, frames):
        """
        Processes each incoming video frame, performs object detection,
        counts persons, and draws results on the frame.
        """
        # Get the current frame as a NumPy array (RGB format).
        frame_data = frames['main'].rw_rgb
        image = frame_data.image  # NumPy array (H, W, C)
        data = frame_data.data    # Metadata dictionary

        # Perform YOLOv8 Inference on the image.
        results = self.model(image, verbose=False)

       
        # 'results[0]' refers to the detections for the first image in the batch.
        # '.boxes.data' contains the raw detection data: [xmin, ymin, xmax, ymax, confidence, class_id]
        # Move to CPU and convert to NumPy for easier manipulation and drawing with OpenCV.
        detections = results[0].boxes.data.cpu().numpy()

        # Create a copy of the image to draw on to avoid modifying the original input.
        output_image = image.copy()

        # Initialize person count for the current frame.
        person_count = 0
        car_count = 0

        # Iterate through detected objects, filter, and draw.
        for detection in detections:
            xmin, ymin, xmax, ymax, confidence, class_id = detection

            # Convert to integer and float types for consistency.
            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
            confidence = float(confidence)
            class_id = int(class_id)


            if confidence >= self.confidence_threshold and (class_id == self.person_class_id or class_id == self.car_class_id):
                if class_id == self.person_class_id:
                    person_count += 1
                else:
                    car_count += 1

                # Prepare label for drawing.
                label = f'{self.class_names_dict[class_id]} {confidence:.2f}' # Use the dictionary for name lookup

                # Define bounding box color (Green in RGB).
                bbox_color = (50, 205, 50)

                # Draw the bounding box rectangle.
                cv2.rectangle(output_image, (xmin, ymin), (xmax, ymax), bbox_color, 2)

                # Draw the label text.
                (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                cv2.rectangle(output_image, (xmin, ymin - text_height - baseline - 5),
                              (xmin + text_width + 5, ymin), (50, 205, 50), -1)
                cv2.putText(output_image, label, (xmin + 2, ymin - baseline - 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)



        count_text = f'Persons:{person_count}, Cars:{car_count}'
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 1
        text_color = (255, 255, 255) # White text
        background_color = (64, 64, 64) # background

        (count_text_width, count_text_height), count_baseline = cv2.getTextSize(count_text, font, font_scale, font_thickness)

        cv2.rectangle(output_image, (10, 10),
                      (10 + count_text_width + 10, 10 + count_text_height + count_baseline + 3),
                      background_color, -1)

        cv2.putText(output_image, count_text, (10, 14 + count_text_height),
                    font, font_scale, text_color, font_thickness, cv2.LINE_AA)

        return Frame(output_image, data, 'RGB')

    def shutdown(self):
        """
        Cleanup method, called when the filter pipeline is shut down.
        """
        print('YOLOv8PersonCounterFilter shutting down')


if __name__ == '__main__':

    Filter.run_multi([
        (VideoIn, dict(sources='file://video.mp4!sync', outputs='tcp://*:5555')),
        (YOLOv8PersonCounterFilter, dict(sources='tcp://localhost:5555', outputs='tcp://*:5552', my_option='PersonCounting')),
        (VideoOut,  dict(sources='tcp://localhost:5552', outputs = 'file://output.mp4')),
    ])



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8PersonCounterFilter setup: config.my_option='PersonCounting'


100%|██████████| 6.25M/6.25M [00:00<00:00, 239MB/s]


YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
YOLOv8 Class Names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toi